In [1]:
import sys

import os

import matplotlib.pyplot as plt

import numpy as np

import healpy as hp

from astropy.io import fits

In [2]:

pyilc_folder ='../'

sys.path.insert(1,pyilc_folder+'pyilc/')

from input import ILCInfo
from wavelets import Wavelets, wavelet_ILC, harmonic_ILC


In [3]:
# this file is where the preprocessed single-frequency maps will be downloaded and saved to.
# they take up about 2.2 Gb; feel free to change this location.

datafolder= pyilc_folder + 'data/' 


In [4]:
# here put the output folder where you want to save the output. 
# this will need to be able to hold XXX Gb.
output_folder = '../output/'
if not os.path.exists(output_folder):
    print("creating folder")
    os.makedirs(output_folder)

In [5]:
nside = 512  # Nside of WMAP
n_freqs = 7  # No. of frequencies that we are using for our analysis

beam_FWHM_arcmin = [32.29, 27.94, 13.08,9.66,7.22,4.92,4.90]  # provide the beam FWHMs. Order needs to always be decreasing. 


def set_freqmapfiles_in_info(ILCInfo, run_num, datafolder='../data/',split='full'):
    assert split in ['full','RH1','RH2']

    ILCInfo.N_side = nside  # Setting the nside to the value used for analysis

    ILCInfo.ELLMAX = 2*nside  # Setting the ell_max to be twise the nside since that is recommended

    ILCInfo.N_freqs = n_freqs  # Setting the no. of frequencies that we will be using for our anlaysis

    ILCInfo.beam_FWHM_arcmin = beam_FWHM_arcmin

    ILCInfo.freq_map_files = []  # Emptying the freq_map_files list, which will contain the directories of the data files. 

    ILCInfo.freq_bp_files = [None] * n_freqs  # Emptying the bandpass files since we dont need to use any bandpasses for our analysis for now

    for xind,x in enumerate([23, 30, 40, 44, 60, 70, 90]):
        ILCInfo.freq_map_files.append('../data/maps/' + str(x) + '_' + split + '_' + str(nside) + '_' + 'run_' + str(run_num) + '.fits')  # Adding the locations of the data files for each run and multiple frequencies to the list. 


In [6]:
def run_full_ILC(info):
    ##########################
    # read in frequency maps
    print("reading in maps and other info...")
    info.read_maps()
    # read in bandpasses
    info.read_bandpasses()
    # read in beams
    info.read_beams()
    #########################
    # construct wavelets
    wv = Wavelets(N_scales=info.N_scales, ELLMAX=info.ELLMAX, tol=1.e-6, taper_width=info.taper_width)
    if info.wavelet_type == 'GaussianNeedlets':
        ell, filts = wv.GaussianNeedlets(FWHM_arcmin=info.GN_FWHM_arcmin)
    elif info.wavelet_type == 'CosineNeedlets': # Fiona added CosineNeedlets
        ell,filts = wv.CosineNeedlets(ellmin = info.ellmin,ellpeaks = info.ellpeaks)
    elif info.wavelet_type == 'TopHatHarmonic':
        ell,filts = wv.TopHatHarmonic(info.ellbins)
    else:
        raise TypeError('unsupported wavelet type')
    # wavelet ILC
    if info.wavelet_type == 'TopHatHarmonic':
        print("converting maps to alms...")
        info.maps2alms()
        print("finding C_ells of maps...")
        info.alms2cls()
        print("doing harmonic ILC...")
        harmonic_ILC(wv, info, resp_tol=info.resp_tol, map_images=False)
    else:
        wavelet_ILC(wv, info, resp_tol=info.resp_tol, map_images=False)
    print("done")
    return

In [8]:
inputfile = pyilc_folder +'input/sample_run_Planck_CMB_HILC.yml'

info_full = ILCInfo(inputfile)

# Setting the total number of runs
run_tot = 12

# Performing multiple runs

for run_num in range(1,run_tot+1): 

    set_freqmapfiles_in_info(info_full, run_num, datafolder)

    info_full.output_dir = output_folder + '/run_' + str(run_num) + '_'

    # print(info_full.freq_map_files)

    # print(len(info_full.freq_map_files))

    # print(info_full.freq_map_files[0])
    # print(hp.get_nside(hp.read_map(info_full.freq_map_files[0])))
    # print(run_num, info_full.N_side)
    # print(type(len(hp.read_map(info_full.freq_map_files[0]))), type(info_full.N_pix))
    # assert len(hp.read_map(info_full.freq_map_files[0])) <= info_full.N_pix, "error"
    # print(hp.get_nside(hp.read_map("../data/maps/23_full_512_run_1.fits")))

    run_full_ILC(info_full)


reading in maps and other info...
converting maps to alms...
finding C_ells of maps...
doing harmonic ILC...
doing main ILC!!
getting none amix
getting none amix
getting none amix
getting none amix
getting none amix
getting none amix
getting none amix
preserved component response failed at wavelet scale 0; these should be zero: [nan] (tol is 0.001)


/home/boorittht/Downloads/pyILC and Commander3/notebooks/../pyilc/wavelets.py:1742: RuntimeWarning: divide by zero encountered in divide
  weights = 1.0/numba_det(np.transpose(Qab_pix,(2,0,1)))[:,None] * np.transpose(tmp3) #N.B. 'weights' here only includes channels that passed beam_thresh criterion
/home/boorittht/Downloads/pyILC and Commander3/notebooks/../pyilc/wavelets.py:1742: RuntimeWarning: invalid value encountered in multiply
  weights = 1.0/numba_det(np.transpose(Qab_pix,(2,0,1)))[:,None] * np.transpose(tmp3) #N.B. 'weights' here only includes channels that passed beam_thresh criterion


NameError: name 'quit' is not defined